In [20]:
import pandas as pd
import psycopg2
from psycopg2 import sql

In [21]:
def create_table_and_ingest_data(db_params, table_name, excel_file_path):
    connection = None
    cursor = None

    try:
        # Read the Excel file into a DataFrame
        df = pd.read_excel(excel_file_path)
        print("Excel file read successfully.")
        print(df.head())  # Display the first few rows of the DataFrame for verification

        # Convert boolean columns to integers
        bool_cols = df.select_dtypes(include='bool').columns
        df[bool_cols] = df[bool_cols].astype(int)

        # Establish a connection to the database
        connection = psycopg2.connect(**db_params)
        cursor = connection.cursor()

        # Drop the table if it exists
        drop_table_query = sql.SQL("DROP TABLE IF EXISTS {table}").format(
            table=sql.Identifier(table_name)
        )
        cursor.execute(drop_table_query)
        connection.commit()

        # Dynamically create table based on DataFrame columns
        create_table_query = sql.SQL(
            "CREATE TABLE {table} ({fields})"
        ).format(
            table=sql.Identifier(table_name),
            fields=sql.SQL(', ').join(
                sql.SQL("{} {}").format(
                    sql.Identifier(col),
                    sql.SQL("TEXT") if df[col].dtype == 'object' else
                    sql.SQL("INTEGER") if df[col].dtype in ['int64', 'bool'] else
                    sql.SQL("FLOAT") if df[col].dtype == 'float64' else
                    sql.SQL("TIMESTAMP") if df[col].dtype == 'datetime64[ns]' else
                    sql.SQL("TEXT")
                ) for col in df.columns
            )
        )
        cursor.execute(create_table_query)
        connection.commit()

        # Build the insert query dynamically based on the DataFrame columns
        insert_query = sql.SQL('INSERT INTO {table} ({fields}) VALUES ({values})').format(
            table=sql.Identifier(table_name),
            fields=sql.SQL(', ').join(map(sql.Identifier, df.columns)),
            values=sql.SQL(', ').join(sql.Placeholder() * len(df.columns))
        )
       
        # Execute insert queries
        for row in df.itertuples(index=False, name=None):
            cursor.execute(insert_query, row)
       
        # Commit the transaction
        connection.commit()

        print("Data ingested successfully.")
    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        # Close the cursor and connection if they were created
        if cursor is not None:
            cursor.close()
        if connection is not None:
            connection.close()

In [22]:
db_params = {
    'dbname': 'amazon',        # Replace with your actual database name
    'user': 'postgres',        # Replace with your actual username
    'password': '1234',    # Replace with your actual password
    'host': 'localhost',            # Adjust if your database server is not local
    'port': '5432'                  # Default PostgreSQL port
}

# Define the table name and the path to the Excel file
table_name = 'amazon_sales'
excel_file_path = r'‪C:\Users\wissen\Downloads\Amazon Sales data 1.xlsx'

# Remove any unwanted Unicode characters
excel_file_path = excel_file_path.replace('\u202a', '').replace('\u202b', '')

# Call the function to create the table and ingest data
create_table_and_ingest_data(db_params, table_name, excel_file_path)

Excel file read successfully.
                              Region                Country        Item Type  \
0              Australia and Oceania                 Tuvalu        Baby Food   
1  Central America and the Caribbean                Grenada           Cereal   
2                             Europe                 Russia  Office Supplies   
3                 Sub-Saharan Africa  Sao Tome and Principe           Fruits   
4                 Sub-Saharan Africa                 Rwanda  Office Supplies   

  Sales Channel Order Priority               Order Date   Order ID  Ship Date  \
0       Offline              H  2022-11-18 11:55:00 UTC  669165933 2010-06-27   
1        Online              C  2023-02-15 08:30:00 UTC  963881480 2012-09-15   
2       Offline              L  2023-01-10 10:45:00 UTC  341417157 2014-05-08   
3        Online              C  2023-03-20 14:20:00 UTC  514321792 2014-07-05   
4       Offline              L  2023-05-05 09:10:00 UTC  115456712 2013-02-06   

  